In [4]:
import arcpy
from arcpy import env
from arcpy.da import *
from arcpy.sa import *
import os
import shutil

In [5]:
arcpy.env.overwriteOutput = True

In [6]:
# 按掩膜提取
def mask_extraction(in_raster,mask_ele,pro_ele):
    ele_pro = arcpy.Describe(pro_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=ele_pro,snapRaster=pro_ele, cellSize=5):
        out_raster = arcpy.sa.ExtractByMask(in_raster, mask_ele, "INSIDE", ele_pro)
        return out_raster
    

def mask_extraction_maxof(in_raster,mask_ele,pro_ele):
    ele_pro = arcpy.Describe(pro_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=ele_pro,snapRaster=pro_ele, cellSize=5,extent="MAXOF"):
        out_raster = arcpy.sa.ExtractByMask(in_raster, mask_ele, "INSIDE", ele_pro)
        return out_raster
def define_projection(input_dataset, target_epsg=4326):
    """
    定义投影为指定的EPSG代码（默认WGS 1984 EPSG: 4326）

    参数:
    input_dataset: 输入的数据集路径
    target_epsg: 目标参考系的EPSG代码，默认值为4326（WGS 1984）
    """
    try:
        # 创建目标空间参考系对象
        spatial_ref = arcpy.SpatialReference(target_epsg)
        
        # 定义投影
        arcpy.DefineProjection_management(input_dataset, spatial_ref)
        
        print(f"投影定义成功: {input_dataset} -> EPSG:{target_epsg}")

    except Exception as e:
        print(f"定义投影时出错: {str(e)}")


def check_projection(input_dataset):
    """
    检查输入数据集是否有定义的坐标系

    参数:
    input_dataset: 输入的数据集路径

    返回:
    True: 如果有定义的坐标系
    False: 如果没有定义的坐标系
    """
    try:
        # 获取数据集的描述信息
        desc = arcpy.Describe(input_dataset)
        
        # 检查是否有定义的坐标系
        if hasattr(desc, "spatialReference") and desc.spatialReference:
            sr = desc.spatialReference
            if sr.name != "Unknown":
                print(f"{input_dataset} 的坐标系为: {sr.name} (EPSG:{sr.factoryCode})")
                return True
            else:
                print(f"{input_dataset} 没有定义坐标系。")
                return False
        else:
            print(f"{input_dataset} 没有定义坐标系。")
            return False

    except Exception as e:
        print(f"检查投影时出错: {str(e)}")
        return False



In [ ]:
# raster_path = r"F:\cache_data\tif_file\DY"  # saga
# raster_path = r"D:\ArcGISProjects\workspace\shbyq\features_data_climate.gdb"  # climate
# raster_path = r"F:\ArcgisData\特征因子数据\climate.gdb"  # download climate
raster_path = r"F:\cache_data\tif_file_texture\dy_20230701_20231031\one_bands"  # sentinel one bands
mask_ele = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb\DEM"
pro_ele = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb\DEM"
save_path = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb"

In [ ]:
env.workspace = raster_path
raster_list = arcpy.ListRasters()
# raster_list.remove('Analytical Hillshading.tif')
# raster_list.remove('DEM')
print(raster_list),len(raster_list)

In [ ]:
# 多个掩膜
for one_raster in raster_list:
    # 定义投影
    # if check_projection(one_raster):
    #     print(1)
    # else:
    #     define_projection(one_raster)
    # # 如果是DEM
    if one_raster == 'dem.tif':
        continue
    print(one_raster)
    result_raster = mask_extraction(one_raster,mask_ele,pro_ele)
    name = str(one_raster).replace(".tif","").replace(" ","").replace("-","")
    result_raster.save(os.path.join(save_path,f"{name}"))
    print(name)

In [10]:
# 单个掩膜
raster_path = r"D:\ArcGISProjects\workspace\shbyq\MyProject.gdb\DY_DZ_result_PolygonToRaster"
mask_ele = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb\DEM"
pro_ele = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb\DEM"
save_path = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb"

In [11]:
env.workspace = save_path

In [12]:
result_raster = mask_extraction_maxof(raster_path,mask_ele,pro_ele)
result_raster.save("DZ")